In [1]:
from osgeo import osr

In [2]:
import os
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "madmex.settings")
import django
django.setup()

In [3]:
import madmex.models

In [4]:
from madmex.models import Object

In [6]:
import datacube
from datetime import datetime

In [7]:
dc = datacube.Datacube(app = 'load_test')

In [39]:
from django.contrib.gis.geos import Polygon, Point
min_lon = -102.0
min_lat = 20
max_lon = -103.0
max_lat = 21
query_polygon = Polygon(((min_lon, max_lat), (max_lon, max_lat), (max_lon, min_lat), (min_lon, min_lat), (min_lon, max_lat)))
test_point = Point(-102.0,20.5)
query_polygon.contains(test_point)

False

In [40]:
sr = dc.load(product='ls8_espa_mexico', 
             longitude=(min_lon, max_lon), 
             latitude=(min_lat, max_lat),
             time=(datetime(2017, 4, 1), datetime(2017, 5, 1)), 
             group_by='solar_day')

In [41]:
sr

<xarray.Dataset>
Dimensions:   (time: 4, x: 3475, y: 3709)
Coordinates:
  * time      (time) datetime64[ns] 2017-04-02T17:17:19.190333 ...
  * y         (y) float64 1.005e+06 1.005e+06 1.005e+06 1.005e+06 1.005e+06 ...
  * x         (x) float64 2.396e+06 2.396e+06 2.396e+06 2.396e+06 2.396e+06 ...
Data variables:
    blue      (time, y, x) int16 567 591 625 583 560 584 645 598 554 533 546 ...
    green     (time, y, x) int16 861 861 910 866 909 891 993 914 845 810 815 ...
    red       (time, y, x) int16 1277 1248 1318 1310 1333 1307 1478 1329 ...
    nir       (time, y, x) int16 2144 2097 2287 2266 2141 2207 2333 2187 ...
    swir1     (time, y, x) int16 3079 2948 3122 3120 2999 3018 3101 3095 ...
    swir2     (time, y, x) int16 2376 2357 2482 2423 2304 2412 2542 2453 ...
    pixel_qa  (time, y, x) int16 322 322 322 322 322 322 322 322 322 322 322 ...
Attributes:
    crs:      PROJCS["unnamed",GEOGCS["WGS 84",DATUM["unknown",SPHEROID["WGS8...

In [42]:
import math
i = 0
for obj in Object.objects.filter(the_geom__intersects=query_polygon):
    i = i + 1
    #print(obj.the_geom.transform(sr.crs.wkt, clone=True))
print(i)

(0.331) SELECT "madmex_object"."id", "madmex_object"."the_geom"::bytea, "madmex_object"."added" FROM "madmex_object" WHERE ST_Intersects("madmex_object"."the_geom", ST_GeomFromEWKB('\x0103000020e6100000010000000500000000000000008059c000000000000035400000000000c059c000000000000035400000000000c059c0000000000000344000000000008059c0000000000000344000000000008059c00000000000003540'::bytea)); args=(<django.contrib.gis.db.backends.postgis.adapter.PostGISAdapter object at 0x7ff32ebe05c0>,)


11941


In [34]:
sr

<xarray.Dataset>
Dimensions:   (time: 2, x: 3475, y: 3709)
Coordinates:
  * time      (time) datetime64[ns] 2017-04-11T17:11:03.885327 ...
  * y         (y) float64 1.005e+06 1.005e+06 1.005e+06 1.005e+06 1.005e+06 ...
  * x         (x) float64 2.5e+06 2.5e+06 2.5e+06 2.5e+06 2.5e+06 2.5e+06 ...
Data variables:
    blue      (time, y, x) int16 298 302 486 654 1136 1315 1261 947 602 627 ...
    green     (time, y, x) int16 782 762 839 876 1694 1956 1874 1270 755 805 ...
    red       (time, y, x) int16 581 550 862 1048 2323 2709 2581 1698 969 ...
    nir       (time, y, x) int16 533 845 2249 2243 3679 4038 3860 2663 1593 ...
    swir1     (time, y, x) int16 201 340 1938 2765 4432 5061 4956 3845 2843 ...
    swir2     (time, y, x) int16 155 210 1199 1972 2981 3460 3427 2817 2115 ...
    pixel_qa  (time, y, x) int16 324 322 322 322 322 322 322 322 322 322 322 ...
Attributes:
    crs:      PROJCS["unnamed",GEOGCS["WGS 84",DATUM["unknown",SPHEROID["WGS8...

In [11]:
print(sr.crs)

PROJCS["unnamed",GEOGCS["WGS 84",DATUM["unknown",SPHEROID["WGS84",6378137,6556752.3141]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["standard_parallel_1",17.5],PARAMETER["standard_parallel_2",29.5],PARAMETER["latitude_of_origin",12],PARAMETER["central_meridian",-102],PARAMETER["false_easting",2500000],PARAMETER["false_northing",0]]


In [38]:
sr.crs.crs_str

'PROJCS["unnamed",GEOGCS["WGS 84",DATUM["unknown",SPHEROID["WGS84",6378137,6556752.3141]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["standard_parallel_1",17.5],PARAMETER["standard_parallel_2",29.5],PARAMETER["latitude_of_origin",12],PARAMETER["central_meridian",-102],PARAMETER["false_easting",2500000],PARAMETER["false_northing",0]]'

In [40]:
dir(sr.crs)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_crs',
 'crs_str',
 'dimensions',
 'epsg',
 'geographic',
 'inverse_flattening',
 'proj',
 'projected',
 'semi_major_axis',
 'semi_minor_axis',
 'units',
 'wkt']

In [54]:
sr.crs.wkt

'PROJCS["unnamed",GEOGCS["WGS 84",DATUM["unknown",SPHEROID["WGS84",6378137,6556752.3141]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["standard_parallel_1",17.5],PARAMETER["standard_parallel_2",29.5],PARAMETER["latitude_of_origin",12],PARAMETER["central_meridian",-102],PARAMETER["false_easting",2500000],PARAMETER["false_northing",0]]'